In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from keras.preprocessing import sequence
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Activation, LSTM, GRU, Embedding, Concatenate, Flatten, BatchNormalization, Conv2D, MaxPooling2D, Convolution2D
from keras.regularizers import l2,l1
from keras import optimizers, layers, regularizers
from keras.optimizers import SGD,Adam,RMSprop
import keras.backend as K

from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
import math
from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
from sklearn.preprocessing import StandardScaler
from scipy import stats
import tensorflow as tf
from keras.engine.topology import Layer
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.model_selection import train_test_split
import math
from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
from sklearn.preprocessing import StandardScaler
from scipy import stats
from keras.backend.tensorflow_backend import set_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import get_session
from sklearn.metrics import brier_score_loss

import nnet_survival
#calibration
import matplotlib.pyplot as plt
import matplotlib
from lifelines import KaplanMeierFitter

#Data process1
import os
from functools import reduce
# Reset Keras Session
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del cox # this is from global space - change this as you need
    except:
        pass

    #print(gc.collect()) # if it's done something you should see a number being outputted

    # use the same config as you used to create the session
    #config = tf.ConfigProto()
    #config.gpu_options.per_process_gpu_memory_fraction = 1
    #config.gpu_options.visible_device_list = "0"
    #set_session(tensorflow.Session(config=config))
    config = ConfigProto()
    config.gpu_options.allow_growth = True
    session = InteractiveSession(config=config)

print("Processing Data-I")

results = pd.DataFrame({'ConcTrain': [], 'ConcVal': [], 'ConcBmTrain': [], 'ConcBmval': []})

clinical = pd.read_csv('data/clinical_data_subsets/clinical_data.csv')
def input(path_omics1, path_omics2, path_omics3):

    #training_list = os.listdir('data/' + path)
    training_list1 = os.listdir('data/' + path_omics1)
    training_list2 = os.listdir('data/' + path_omics2)
    training_list3 = os.listdir('data/' + path_omics3)

    # training_list2.to_csv(t2.csv)
    # training_list3.to_csv(t3.csv)
    # training_list1.to_csv(t1.csv)
    # print(training_list2)
    # print(training_list3)
    # print(training_list1)
    
    #training_list = set(training_list1) & set(training_list2) & set(training_list3)
    training_list = reduce(np.intersect1d, (training_list1, training_list3, training_list2))
    print(len(training_list1), len(training_list2), len(training_list3))
    print(len(training_list))
    #training_list = np.intersect1d(training_list1, training_list2, training_list3)

    #folder  = 'data/' + path
    training_list.sort()    

    training_list = np.asarray(training_list, dtype=object)
    shape = (len(training_list), 122, 122, 1)
    shape_mirna = (len(training_list), 42, 42, 1)
    print(shape)
    #input_shape = (122, 122, 1)

    dataset1 = np.ndarray(shape=shape,dtype=np.float32)
    dataset2 = np.ndarray(shape=shape,dtype=np.float32)
    dataset3 = np.ndarray(shape=shape_mirna,dtype=np.float32)

    #Reference: https://www.kaggle.com/lgmoneda/data-augmentation-regression

    i = 0
    for item in training_list:
        img1 = load_img("data/" + path_omics1 + '/' + item, target_size=(122,122), color_mode='grayscale')  # this is a PIL image
        img2 = load_img("data/" + path_omics2 + '/' + item, target_size=(122,122), color_mode='grayscale')  # this is a PIL image
        img3 = load_img("data/" + path_omics3 + '/' + item, target_size=(42,42), color_mode='grayscale')  # this is a PIL image
        # Convert to Numpy Array
        x1 = img_to_array(img1) 
        x2 = img_to_array(img2)  
        x3 = img_to_array(img3)
        #x = x.reshape((3, 120, 160))
        # Normalize
        #x = (x - 128.0) / 128.0
        dataset1[i] = x1
        dataset2[i] = x2
        dataset3[i] = x3
        i += 1
        if i % 100 == 0:
            print("%d images to array" % i)
    print("All images to array!")

    return dataset1, dataset2, dataset3, training_list

dataset_meth, dataset_mrna, dataset_mirna, training_list = input('training_data_meth', 'training_data_mrna', 'training_data_mirna')

#Data process2
from tqdm import tqdm
print("Processing Data-II")
sample = []
t = []
f = []
age = []

for list in tqdm(training_list):
    for i in range(len(clinical)):
        if clinical.iloc[i]['sample'] + '.png' == str(list):
            p_id = clinical.iloc[i]['sample']
            time = clinical.iloc[i]['os_time']
            status = clinical.iloc[i]['vital_status']
            a = clinical.iloc[i]['age']

            sample.append(p_id)
            t.append(time)
            f.append(status)
            age.append(a)
            continue
        else:
            pass
t  = np.asarray(t)
f  = np.asarray(f)
sample  = np.asarray(sample)
age = np.asarray(age)
#clinical.iloc[1]['sample']

Using TensorFlow backend.
Processing Data-I
458 515 450
446
(446, 122, 122, 1)
100 images to array
200 images to array
  0%|          | 0/446 [00:00<?, ?it/s]300 images to array
400 images to array
All images to array!
Processing Data-II
100%|██████████| 446/446 [00:23<00:00, 19.26it/s]


In [2]:
for i in range(1,50):
    #random seed
    from numpy.random import seed
    seed(123)
    import tensorflow as tf
    tf.random.set_random_seed(123)

    #GPUconfig
    from tensorflow.compat.v1 import ConfigProto
    from tensorflow.compat.v1 import InteractiveSession
    config = ConfigProto()
    config.gpu_options.allow_growth = True
    session = InteractiveSession(config=config)

    breaks=np.arange(0.,365.*10,365./4)
    n_intervals=len(breaks)-1
    y_train_array = nnet_survival.make_surv_array(t,f,breaks)

    # random  = random_state for test train split
    indices = range(len(f))
    random = i
    split_ratio = 0.2
    batch_size = 8

    X_train_mrna, X_test_mrna, y_train, y_test, ind_train_1, ind_test_1 = train_test_split(dataset_mrna, y_train_array,indices, test_size=split_ratio, random_state=random)
    X_train_meth, X_test_meth, y_train, y_test, ind_train_2, ind_test_2 = train_test_split(dataset_meth, y_train_array,indices, test_size=split_ratio, random_state=random)
    X_train_mirna, X_test_mirna, y_train, y_test, ind_train_2, ind_test_2 = train_test_split(dataset_mirna, y_train_array,indices, test_size=split_ratio, random_state=random)
    #age_train, age_test, placeholder_train, placeholder_test, ind_train, ind_test = train_test_split(age, y_train_array,indices, test_size=split_ratio, random_state=random)

    #----------------------------------------- keras functional model for multi-omics data --------------------------
    #input1

    input_1 = Input(shape = (122,122,1))
    mrna_conv_1   = Convolution2D(256, (3, 3), kernel_initializer='glorot_normal')(input_1)
    mrna_bn_1     = BatchNormalization()(mrna_conv_1)
    mrna_act_1    = Activation('relu')(mrna_bn_1)
    mrna_pool_1   = MaxPooling2D(pool_size = (2,2))(mrna_act_1)

    mrna_conv_2   = Convolution2D(256, (3, 3), kernel_initializer='glorot_normal')(mrna_pool_1)
    mrna_bn_2     = BatchNormalization()(mrna_conv_2)
    mrna_act_2    = Activation('relu')(mrna_bn_2)
    mrna_pool_2   = MaxPooling2D(pool_size = (2,2))(mrna_act_2)

    flat_1 = Flatten()(mrna_pool_2)

    ################################################# convolutiona
    #input2
    input_2 = Input(shape = (122,122,1))
    meth_conv_1   = Convolution2D(256, (3, 3), kernel_initializer='glorot_normal')(input_2)
    meth_bn_1     = BatchNormalization()(meth_conv_1)
    meth_act_1    = Activation('relu')(meth_bn_1)
    meth_pool_1   = MaxPooling2D(pool_size = (2,2))(meth_act_1)

    meth_conv_2   = Convolution2D(256, (3, 3), kernel_initializer='glorot_normal')(meth_pool_1)
    meth_bn_2     = BatchNormalization()(meth_conv_2)
    meth_act_2    = Activation('relu')(meth_bn_2)
    meth_pool_2   = MaxPooling2D(pool_size = (2,2))(meth_act_2)

    flat_2 = Flatten()(meth_pool_2)

    #input3
    input_3 = Input(shape = (42,42,1))
    mirna_conv_1   = Convolution2D(256, (3, 3), kernel_initializer='glorot_normal')(input_3)
    mirna_bn_1     = BatchNormalization()(mirna_conv_1)
    mirna_act_1    = Activation('relu')(mirna_bn_1)
    mirna_pool_1   = MaxPooling2D(pool_size = (2,2))(mirna_act_1)

    mirna_conv_2   = Convolution2D(256, (3, 3), kernel_initializer='glorot_normal')(mirna_pool_1)
    mirna_bn_2     = BatchNormalization()(mirna_conv_2)
    mirna_act_2    = Activation('relu')(mirna_bn_2)
    mirna_pool_2   = MaxPooling2D(pool_size = (2,2))(mirna_act_2)

    flat_3 = Flatten()(mirna_pool_2)
    #input_2_dense  = Dense(1, activation = 'relu',kernel_initializer='glorot_normal')(input_2)
    #flat_2 = Flatten()(input_2_dense)

    ################################################Phenotypes##################
    #input_3 = Input(shape=(1, ))

    concat = Concatenate()([flat_1,flat_2,flat_3])

    dense_1 = Dense(256, activation = 'relu',kernel_initializer='glorot_normal')(concat)
    #dense_2 = Dense(128, activation = 'relu',kernel_initializer='glorot_normal')(dense_1)
    #dense_3 = Dense(1, use_bias=0, kernel_initializer='zeros')(dense_2)

    prop_hazards=1
    if prop_hazards:
        #dense_3 = Dense(1, use_bias=0, kernel_initializer='zeros')(dense_2)
        
        dense_3 = Dense(1, use_bias=0, kernel_initializer='zeros')(dense_1)
        output  = nnet_survival.PropHazards(n_intervals)(dense_3)
    else:
        output = Dense(n_intervals, kernel_initializer='zeros', bias_initializer='zeros', activation = 'sigmoid')(dense_2)

    cox = Model(inputs=[input_1,input_2, input_3], outputs=[output])

    print(cox.summary())

    from tensorflow import keras
    #os.environ["PATH"] += os.pathsep + '/home/dell15/KING/Work_ubuntu/Projects/MO-int/king_env/lib/python3.6/site-packages/graphviz'
    keras.utils.plot_model(cox, 'multi_input_and_output_model.png', show_shapes=True)

    #Gradient descent
    sgd  = SGD(lr=0.00001, decay=1e-6, momentum=0.9, nesterov=True)
    cox.compile(loss=nnet_survival.surv_likelihood(n_intervals), optimizer=sgd)
    #Early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=25)
    filepath='checkpoints/mrna_meth_mirna/one_dense_weights-improvement-' + str(i) + '.hdf5'
    model_checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

    # for functional models of multiomics
    history=cox.fit([X_train_mrna, X_train_meth, X_train_mirna], y_train, batch_size=batch_size, epochs=200, verbose=1, validation_data=([X_test_mrna,X_test_meth, X_test_mirna],y_test), callbacks=[early_stopping,model_checkpoint])

    #----------------------------------------------------------------------------- for functional model ----------------
    #Training set results
    y_pred=cox.predict([X_train_mrna,X_train_meth, X_train_mirna],verbose=0, batch_size = batch_size)
    #validation set results
    y_pred_valid=cox.predict([X_test_mrna,X_test_meth, X_test_mirna],verbose=0, batch_size = batch_size)

    #-------------------------------------------- run for functional model------------------------------------
    # load the saved model
    saved_model = load_model('checkpoints/mrna_meth_mirna/one_dense_weights-improvement-' + str(i) + '.hdf5', custom_objects={'PropHazards': nnet_survival.PropHazards(n_intervals), 'loss': nnet_survival.surv_likelihood(n_intervals)})
    #Training set results---------------------------- on best model for the process-------------
    y_pred_bm=saved_model.predict([X_train_mrna,X_train_meth, X_train_mirna],verbose=0, batch_size = batch_size)
    #validation set results
    y_pred_valid_bm=saved_model.predict([X_test_mrna,X_test_meth, X_test_mirna],verbose=0, batch_size = batch_size)

    T_train, T_test, F_train, F_test, TF_ind_train, TF_ind_test = train_test_split(t, f,indices, test_size=split_ratio, random_state=random)
    one_year_survival_prob_train = np.cumprod(y_pred[:,0:np.nonzero(breaks>365)[0][0]], axis=1)[:,-1]
    one_year_survival_prob_valid = np.cumprod(y_pred_valid[:,0:np.nonzero(breaks>365)[0][0]], axis=1)[:,-1]
    one_yr_median_train = np.median(one_year_survival_prob_train)
    one_yr_median_valid = np.median(one_year_survival_prob_valid)
    print("Mediantrain | MedianTest:", one_yr_median_train,one_yr_median_valid)

    # Concordance-index------------------------------------------------------------------- 5yr-------------
    fiveyr_surv=np.cumprod(y_pred[:,0:np.nonzero(breaks>1825)[0][0]], axis=1)[:,-1]
    ConcTrain = concordance_index(T_train,fiveyr_surv,F_train)
    print("ConcTrain:", concordance_index(T_train,fiveyr_surv,F_train))

    # Concordance-index----- for validation set
    fiveyr_surv_valid=np.cumprod(y_pred_valid[:,0:np.nonzero(breaks>1825)[0][0]], axis=1)[:,-1]
    ConcVal = concordance_index(T_test,fiveyr_surv_valid,F_test)
    print('ConcVal:', concordance_index(T_test,fiveyr_surv_valid,F_test))

    # Concordance-index----------------------------- 5yr-----------------------------------best model----------
    fiveyr_surv_bm=np.cumprod(y_pred_bm[:,0:np.nonzero(breaks>1825)[0][0]], axis=1)[:,-1]
    ConcBmTrain = concordance_index(T_train,fiveyr_surv_bm,F_train)
    print('ConcBmTrain',concordance_index(T_train,fiveyr_surv_bm,F_train))

    # Concordance-index----- for validation set---------------------------------------------best model---------
    fiveyr_surv_valid_bm=np.cumprod(y_pred_valid_bm[:,0:np.nonzero(breaks>1825)[0][0]], axis=1)[:,-1]
    ConcBmval = concordance_index(T_test,fiveyr_surv_valid_bm,F_test)
    print('ConcBmval',concordance_index(T_test,fiveyr_surv_valid_bm,F_test))

    #calculates True cumulative probabilitiy of 5yr survival
    brier_true_train = np.cumprod(y_train[:,0:np.nonzero(breaks>1825)[0][0]], axis=1)[:,-1]
    brier_true_test = np.cumprod(y_test[:,0:np.nonzero(breaks>1825)[0][0]], axis=1)[:,-1]
    brier_true_all = np.cumprod(y_train_array[:,0:np.nonzero(breaks>1825)[0][0]], axis=1)[:,-1]
    # # plot impact of brier for single forecasts for 5yr duration
    train_brier, test_brier = brier_score_loss(brier_true_train, fiveyr_surv), brier_score_loss(brier_true_test, fiveyr_surv_valid)
    print('Brier scores:', train_brier, test_brier)

    df = {'ConcTrain': ConcTrain, 'ConcVal': ConcVal, 'ConcBmTrain': ConcBmTrain, 'ConcBmval':ConcBmval, 'BrierTrain': train_brier, 'BrierTest': test_brier}
    results = results.append(df, ignore_index=True)

    results.to_csv('mrna_meth_mirna/one_dense/res_' + str(i) + '.csv')


    kmf = KaplanMeierFitter()
    #matplotlib.style.use('default')
    actual = []
    predicted = []
    plt.figure(figsize=(12,5))
    plt.subplot(1, 2, 1)
    days_plot = 365*9
    for j in range(2):
        if j==0:
            kmf.fit(T_train[one_year_survival_prob_train >= one_yr_median_train ], event_observed=F_train[one_year_survival_prob_train >= one_yr_median_train])
            kmf.plot()
            #plt.plot(kmf.survival_function_.index.values, kmf.survival_function_.KM_estimate,ls='solid',c='C'+str(i))
        #pred_surv=np.mean(np.cumprod(y_pred_valid[F_test==i,:], axis=1),axis=0)
        #plt.plot(breaks,np.concatenate(([1],pred_surv)),ls='--',c='C'+str(i))
        else:
            kmf.fit(T_train[one_year_survival_prob_train < one_yr_median_train ], event_observed=F_train[one_year_survival_prob_train < one_yr_median_train])
            kmf.plot()
            #plt.plot(kmf.survival_function_.index.values, kmf.survival_function_.KM_estimate,ls='solid',c='C'+str(i))

    N1 = ' N= ' + str(len(T_train[one_year_survival_prob_train >= one_yr_median_train ]))
    N2 = ' N= ' + str(len(T_train[one_year_survival_prob_train < one_yr_median_train ]))

    plt.xticks(np.arange(0, days_plot, 365))
    plt.yticks(np.arange(0, 1.125, 0.125))
    plt.xlim([0,days_plot])
    plt.ylim([0,1])
    plt.xlabel('Follow-up time (days)')
    plt.ylabel('Probability of survival')
    plt.legend(['Low Risk Individuals' + N1 ,'High Risk Individuals' + N2 ])
    #	'1: Actual','1: Predicted')
    # 	'2: Actual','2: Predicted',
    # 	'3: Actual','3: Predicted',
    # 	'4: Actual','4: Predicted'])
    plt.title('Train set Kaplan-Meier Curves', fontweight = 'bold')
    plt.grid()
    #plt.show()
    plt.subplot(1, 2, 2)
    days_plot = 365*9
    for j in range(2):
        if j==0:
            kmf.fit(T_test[one_year_survival_prob_valid >= one_yr_median_valid ], event_observed=F_test[one_year_survival_prob_valid >= one_yr_median_valid])
            kmf.plot()
            #plt.plot(kmf.survival_function_.index.values, kmf.survival_function_.KM_estimate,ls='solid',c='C'+str(i))
        #pred_surv=np.mean(np.cumprod(y_pred_valid[F_test==i,:], axis=1),axis=0)
        #plt.plot(breaks,np.concatenate(([1],pred_surv)),ls='--',c='C'+str(i))
        else:
            kmf.fit(T_test[one_year_survival_prob_valid < one_yr_median_valid ], event_observed=F_test[one_year_survival_prob_valid < one_yr_median_valid])
            kmf.plot()
            #plt.plot(kmf.survival_function_.index.values, kmf.survival_function_.KM_estimate,ls='solid',c='C'+str(i))

    N1 = ' N= ' + str(len(T_test[one_year_survival_prob_valid >= one_yr_median_valid ]))
    N2 = ' N= ' + str(len(T_test[one_year_survival_prob_valid < one_yr_median_valid ]))
    plt.xticks(np.arange(0, days_plot, 365))
    plt.yticks(np.arange(0, 1.125, 0.125))
    plt.xlim([0,days_plot])
    plt.ylim([0,1])
    plt.xlabel('Follow-up time (days)')
    plt.ylabel('Probability of survival')
    plt.legend(['Low Risk Individuals' + N1 ,'High Risk Individuals' + N2 ])
    #	'1: Actual','1: Predicted')
    # 	'2: Actual','2: Predicted',
    # 	'3: Actual','3: Predicted',
    # 	'4: Actual','4: Predicted'])
    plt.title('Test set Kaplan-Meier Curves', fontweight = 'bold')
    plt.grid()
    #plt.show()
    plt.savefig('mrna_meth_mirna/one_dense/plots/KP_' + str(i) + '.png', format = 'png', dpi = 1200, bbox_inches='tight')
    plt.clf()
    reset_keras()

#results.to_csv('res_final.csv')

055: val_loss did not improve from 1.34567
Epoch 56/200
356/356 [==============================] - 4s 12ms/step - loss: 1.3552 - val_loss: 1.4100

Epoch 00056: val_loss did not improve from 1.34567
Epoch 57/200
356/356 [==============================] - 4s 12ms/step - loss: 1.3558 - val_loss: 1.3765

Epoch 00057: val_loss did not improve from 1.34567
Epoch 58/200
356/356 [==============================] - 4s 12ms/step - loss: 1.3438 - val_loss: 1.4122

Epoch 00058: val_loss did not improve from 1.34567
Epoch 59/200
356/356 [==============================] - 4s 12ms/step - loss: 1.3347 - val_loss: 1.3809

Epoch 00059: val_loss did not improve from 1.34567
Epoch 60/200
356/356 [==============================] - 4s 12ms/step - loss: 1.3366 - val_loss: 1.3497

Epoch 00060: val_loss did not improve from 1.34567
Epoch 61/200
356/356 [==============================] - 4s 12ms/step - loss: 1.3449 - val_loss: 1.3563

Epoch 00061: val_loss did not improve from 1.34567
Epoch 62/200
356/356 [=====

<Figure size 864x360 with 0 Axes>

<Figure size 864x360 with 0 Axes>

<Figure size 864x360 with 0 Axes>

<Figure size 864x360 with 0 Axes>

...

<Figure size 864x360 with 0 Axes>

<Figure size 864x360 with 0 Axes>

<Figure size 864x360 with 0 Axes>

<Figure size 864x360 with 0 Axes>

<Figure size 864x360 with 0 Axes>

In [ ]:
print(cox.summary())

In [ ]:
# age_train, age_test, placeholder_train, placeholder_test, ind_train, ind_test = train_test_split(age, y_train_array,indices, test_size=split_ratio, random_state=random)

In [ ]:
X_train_mrna.shape, X_train_meth.shape

In [ ]:
# from sklearn.model_selection import StratifiedKFold-------------------------- not used because of errors; dimension of predictors not compatible
# # define 10-fold cross validation 
# kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=33)
#train, test  = kfold.split(dataset, y_train_array)

In [ ]:
#----------------------------------------- keras functional model for multi-omics data --------------------------

input_1 = Input(shape = (122,122,1))
conv_1   = Convolution2D(256, (3, 3), kernel_initializer='glorot_normal')(input_1)
bn_1     = BatchNormalization()(conv_1)
act_1    = Activation('relu')(bn_1)
pool_1   = MaxPooling2D(pool_size = (2,2))(act_1)

conv_2   = Convolution2D(256, (3, 3), kernel_initializer='glorot_normal')(pool_1)
bn_2     = BatchNormalization()(conv_2)
act_2    = Activation('relu')(bn_2)
pool_2   = MaxPooling2D(pool_size = (2,2))(act_2)

flat_1 = Flatten()(pool_2)

################################################# convolutional above | phenotype variables below

input_2 = Input(shape=(1, ))
#input_2_dense  = Dense(1, activation = 'relu',kernel_initializer='glorot_normal')(input_2)
#flat_2 = Flatten()(input_2_dense)


concat = Concatenate()([flat_1,input_2])

#dense_1 = Dense(256, activation = 'relu',kernel_initializer='glorot_normal')(concat)
dense_2 = Dense(128, activation = 'relu',kernel_initializer='glorot_normal')(concat)
#dense_3 = Dense(1, use_bias=0, kernel_initializer='zeros')(dense_2)

prop_hazards=1
if prop_hazards:
    dense_3 = Dense(1, use_bias=0, kernel_initializer='zeros')(dense_2)
    output  = nnet_survival.PropHazards(n_intervals)(dense_3)
else:
    output = Dense(n_intervals, kernel_initializer='zeros', bias_initializer='zeros', activation = 'sigmoid')(dense_2)

cox = Model(inputs=[input_1,input_2], outputs=[output])



In [ ]:
#----------------------------------------- keras functional model for multi-omics data --------------------------

input_1 = Input(shape = (122,122,1))
mrna_conv_1   = Convolution2D(256, (3, 3), kernel_initializer='glorot_normal')(input_1)
mrna_bn_1     = BatchNormalization()(mrna_conv_1)
mrna_act_1    = Activation('relu')(mrna_bn_1)
mrna_pool_1   = MaxPooling2D(pool_size = (2,2))(mrna_act_1)

mrna_conv_2   = Convolution2D(256, (3, 3), kernel_initializer='glorot_normal')(mrna_pool_1)
mrna_bn_2     = BatchNormalization()(mrna_conv_2)
mrna_act_2    = Activation('relu')(mrna_bn_2)
mrna_pool_2   = MaxPooling2D(pool_size = (2,2))(mrna_act_2)

flat_1 = Flatten()(mrna_pool_2)

################################################# convolutional above | phenotype variables below

input_2 = Input(shape = (122,122,1))
conv_1   = Convolution2D(256, (3, 3), kernel_initializer='glorot_normal')(input_2)
bn_1     = BatchNormalization()(conv_1)
act_1    = Activation('relu')(bn_1)
pool_1   = MaxPooling2D(pool_size = (2,2))(act_1)

conv_2   = Convolution2D(256, (3, 3), kernel_initializer='glorot_normal')(pool_1)
bn_2     = BatchNormalization()(conv_2)
act_2    = Activation('relu')(bn_2)
pool_2   = MaxPooling2D(pool_size = (2,2))(act_2)

flat_2 = Flatten()(pool_2)
#input_2_dense  = Dense(1, activation = 'relu',kernel_initializer='glorot_normal')(input_2)
#flat_2 = Flatten()(input_2_dense)


concat = Concatenate()([flat_1,flat_2])

dense_1 = Dense(256, activation = 'relu',kernel_initializer='glorot_normal')(concat)
dense_2 = Dense(128, activation = 'relu',kernel_initializer='glorot_normal')(dense_1)
#dense_3 = Dense(1, use_bias=0, kernel_initializer='zeros')(dense_2)

prop_hazards=1
if prop_hazards:
    dense_3 = Dense(1, use_bias=0, kernel_initializer='zeros')(dense_2)
    output  = nnet_survival.PropHazards(n_intervals)(dense_3)
else:
    output = Dense(n_intervals, kernel_initializer='zeros', bias_initializer='zeros', activation = 'sigmoid')(dense_2)

cox = Model(inputs=[input_1,input_2], outputs=[output])

In [ ]:
# from keras.utils import plot_model
# plot_model(cox, to_file='model.png')

In [ ]:
# #keras Sequential API example https://theailearner.com/2019/01/25/multi-input-and-multi-output-models-in-keras/
# # feature extraction from gray scale image
# inputs = Input(shape = (28,28,1))
 
# conv1 = Conv2D(16, (3,3), activation = 'relu', padding = "SAME")(inputs)
# pool1 = MaxPooling2D(pool_size = (2,2), strides = 2)(conv1)
# conv2 = Conv2D(32, (3,3), activation = 'relu', padding = "SAME")(pool1)
# pool2 = MaxPooling2D(pool_size = (2,2), strides = 2)(conv2)
# flat_1 = Flatten()(pool2)
 
# # feature extraction from RGB image
# inputs_2 = Input(shape = (28,28,3))
 
# conv1_2 = Conv2D(16, (3,3), activation = 'relu', padding = "SAME")(inputs_2)
# pool1_2 = MaxPooling2D(pool_size = (2,2), strides = 2)(conv1_2)
# conv2_2 = Conv2D(32, (3,3), activation = 'relu', padding = "SAME")(pool1_2)
# pool2_2 = MaxPooling2D(pool_size = (2,2), strides = 2)(conv2_2)
# flat_2 = Flatten()(pool2_2)
 
# # concatenate both feature layers and define output layer after some dense layers
# concat = concatenate([flat_1,flat_2])
# dense1 = Dense(512, activation = 'relu')(concat)
# dense2 = Dense(128, activation = 'relu')(dense1)
# dense3 = Dense(32, activation = 'relu')(dense2)
# output = Dense(10, activation = 'softmax')(dense3)
 
# # create model with two inputs
# model = Model([inputs,inputs_2], output)

In [ ]:
#Gradient descent
sgd  = SGD(lr=0.00001, decay=1e-6, momentum=0.9, nesterov=True)
cox.compile(loss=nnet_survival.surv_likelihood(n_intervals), optimizer=sgd)
#Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=20)
filepath='checkpoints/weights-improvement-{epoch:02d}-{val_loss:.4f}.hdf5'
model_checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [ ]:
# for sequential models
#history=cox.fit(X_train, y_train, batch_size=batch_size, epochs=200, verbose=1, validation_data=(X_test,y_test), callbacks=[early_stopping,model_checkpoint])

In [ ]:
#history=cox.fit(X_train, y_train, batch_size=batch_size, epochs=300, verbose=1, validation_data=(X_test,y_test), callbacks=[early_stopping,model_checkpoint], initial_epoch=200)

In [ ]:
#initial_epoch=10

In [ ]:
#history=cox.fit(dataset, y_train_array, batch_size=batch_size, epochs=100, verbose=1, validation_split=0.2)

In [ ]:
fig = plt.figure()
#plt.plot(hist.history['val_loss'])
#plt.plot(hist.history['loss'])
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("model loss")
plt.ylabel("Negative Log Likelihood Loss")
plt.xlabel("Epoch")
plt.legend(["Training Loss","Validation Loss"])
#"Training Loss","Validation Loss"]))
plt.show()

In [ ]:
# ############################################################----------------------------run for sequential-
#Training set results
y_pred=cox.predict_proba(X_train,verbose=0, batch_size = batch_size)
#validation set results
y_pred_valid=cox.predict_proba(X_test,verbose=0, batch_size = batch_size)


In [ ]:
#----------------------------------------------------------------------------- for functional model ----------------
#Training set results
y_pred=cox.predict([X_train_mrna,X_train_meth],verbose=0, batch_size = batch_size)
#validation set results
y_pred_valid=cox.predict([X_test_mrna,X_test_meth],verbose=0, batch_size = batch_size)

In [ ]:
y_pred

In [ ]:
y_pred_valid

In [ ]:
#-------------------------------------------- run for sequential model------------------------------------
# load the saved model
saved_model = load_model('checkpoints/weights-improvement-32-1.5237.hdf5', custom_objects={'PropHazards': nnet_survival.PropHazards(n_intervals), 'loss': nnet_survival.surv_likelihood(n_intervals)})
#Training set results---------------------------- on best model for the process-------------
y_pred_bm=saved_model.predict_proba(X_train,verbose=0, batch_size = batch_size)
#validation set results
y_pred_valid_bm=saved_model.predict_proba(X_test,verbose=0, batch_size = batch_size)

In [ ]:
T_train, T_test, F_train, F_test, TF_ind_train, TF_ind_test = train_test_split(t, f,indices, test_size=split_ratio, random_state=random)
one_year_survival_prob_train = np.cumprod(y_pred[:,0:np.nonzero(breaks>365)[0][0]], axis=1)[:,-1]
one_year_survival_prob_valid = np.cumprod(y_pred_valid[:,0:np.nonzero(breaks>365)[0][0]], axis=1)[:,-1]
one_yr_median_train = np.median(one_year_survival_prob_train)
one_yr_median_valid = np.median(one_year_survival_prob_valid)

In [ ]:
one_yr_median_train,one_yr_median_valid

In [ ]:
# Concordance-index------------------------------------------------------------------- 5yr-------------
fiveyr_surv=np.cumprod(y_pred[:,0:np.nonzero(breaks>1825)[0][0]], axis=1)[:,-1]
print(concordance_index(T_train,fiveyr_surv,F_train))

# Concordance-index----- for validation set
fiveyr_surv_valid=np.cumprod(y_pred_valid[:,0:np.nonzero(breaks>1825)[0][0]], axis=1)[:,-1]
print(concordance_index(T_test,fiveyr_surv_valid,F_test))

# Concordance-index----------------------------- 5yr-----------------------------------best model----------
fiveyr_surv_bm=np.cumprod(y_pred_bm[:,0:np.nonzero(breaks>1825)[0][0]], axis=1)[:,-1]
print(concordance_index(T_train,fiveyr_surv_bm,F_train))

# Concordance-index----- for validation set---------------------------------------------best model---------
fiveyr_surv_valid_bm=np.cumprod(y_pred_valid_bm[:,0:np.nonzero(breaks>1825)[0][0]], axis=1)[:,-1]
print(concordance_index(T_test,fiveyr_surv_valid_bm,F_test))

In [ ]:
#calibration
import matplotlib.pyplot as plt
import matplotlib
from lifelines import KaplanMeierFitter
#plt.figure(figsize=(10,5))
#plt.subplot(1, 2, 2)


In [ ]:
kmf = KaplanMeierFitter()
#matplotlib.style.use('default')
actual = []
predicted = []
plt.figure(figsize=(12,5))
plt.subplot(1, 2, 1)
days_plot = 365*9
for i in range(2):
    if i==0:
        kmf.fit(T_train[one_year_survival_prob_train >= one_yr_median_train ], event_observed=F_train[one_year_survival_prob_train >= one_yr_median_train])
        kmf.plot()
        #plt.plot(kmf.survival_function_.index.values, kmf.survival_function_.KM_estimate,ls='solid',c='C'+str(i))
	#pred_surv=np.mean(np.cumprod(y_pred_valid[F_test==i,:], axis=1),axis=0)
	#plt.plot(breaks,np.concatenate(([1],pred_surv)),ls='--',c='C'+str(i))
    else:
        kmf.fit(T_train[one_year_survival_prob_train < one_yr_median_train ], event_observed=F_train[one_year_survival_prob_train < one_yr_median_train])
        kmf.plot()
        #plt.plot(kmf.survival_function_.index.values, kmf.survival_function_.KM_estimate,ls='solid',c='C'+str(i))

N1 = ' N= ' + str(len(T_train[one_year_survival_prob_train >= one_yr_median_train ]))
N2 = ' N= ' + str(len(T_train[one_year_survival_prob_train < one_yr_median_train ]))

plt.xticks(np.arange(0, days_plot, 365))
plt.yticks(np.arange(0, 1.125, 0.125))
plt.xlim([0,days_plot])
plt.ylim([0,1])
plt.xlabel('Follow-up time (days)')
plt.ylabel('Probability of survival')
plt.legend(['Low Risk Individuals' + N1 ,'High Risk Individuals' + N2 ])
#	'1: Actual','1: Predicted')
# 	'2: Actual','2: Predicted',
# 	'3: Actual','3: Predicted',
# 	'4: Actual','4: Predicted'])
plt.title('Train set Kaplan-Meier Curves', fontweight = 'bold')
plt.grid()
#plt.show()
plt.subplot(1, 2, 2)
days_plot = 365*9
for i in range(2):
    if i==0:
        kmf.fit(T_test[one_year_survival_prob_valid >= one_yr_median_valid ], event_observed=F_test[one_year_survival_prob_valid >= one_yr_median_valid])
        kmf.plot()
        #plt.plot(kmf.survival_function_.index.values, kmf.survival_function_.KM_estimate,ls='solid',c='C'+str(i))
	#pred_surv=np.mean(np.cumprod(y_pred_valid[F_test==i,:], axis=1),axis=0)
	#plt.plot(breaks,np.concatenate(([1],pred_surv)),ls='--',c='C'+str(i))
    else:
        kmf.fit(T_test[one_year_survival_prob_valid < one_yr_median_valid ], event_observed=F_test[one_year_survival_prob_valid < one_yr_median_valid])
        kmf.plot()
        #plt.plot(kmf.survival_function_.index.values, kmf.survival_function_.KM_estimate,ls='solid',c='C'+str(i))

N1 = ' N= ' + str(len(T_test[one_year_survival_prob_valid >= one_yr_median_valid ]))
N2 = ' N= ' + str(len(T_test[one_year_survival_prob_valid < one_yr_median_valid ]))
plt.xticks(np.arange(0, days_plot, 365))
plt.yticks(np.arange(0, 1.125, 0.125))
plt.xlim([0,days_plot])
plt.ylim([0,1])
plt.xlabel('Follow-up time (days)')
plt.ylabel('Probability of survival')
plt.legend(['Low Risk Individuals' + N1 ,'High Risk Individuals' + N2 ])
#	'1: Actual','1: Predicted')
# 	'2: Actual','2: Predicted',
# 	'3: Actual','3: Predicted',
# 	'4: Actual','4: Predicted'])
plt.title('Test set Kaplan-Meier Curves', fontweight = 'bold')
plt.grid()
#plt.show()
plt.savefig('dummy.png', format = 'png', dpi = 1200, bbox_inches='tight')

In [ ]:
#Full DATASET results
y_pred_overall=cox.predict([dataset_mrna,dataset_meth],verbose=0, batch_size = batch_size)

one_year_survival_prob_all = np.cumprod(y_pred_overall[:,0:np.nonzero(breaks>365)[0][0]], axis=1)[:,-1]
one_yr_median_all = np.median(one_year_survival_prob_all)

plt.figure(figsize=(8,6))
#plt.subplot(1, 2, 1)
days_plot = 365*9
for i in range(2):
    if i==0:
        kmf.fit(t[one_year_survival_prob_all >= one_yr_median_all ], event_observed=f[one_year_survival_prob_all >= one_yr_median_all])
        kmf.plot()
        #plt.plot(kmf.survival_function_.index.values, kmf.survival_function_.KM_estimate,ls='solid',c='C'+str(i))
	#pred_surv=np.mean(np.cumprod(y_pred_valid[F_test==i,:], axis=1),axis=0)
	#plt.plot(breaks,np.concatenate(([1],pred_surv)),ls='--',c='C'+str(i))
    else:
        kmf.fit(t[one_year_survival_prob_all < one_yr_median_all ], event_observed=f[one_year_survival_prob_all < one_yr_median_all])
        kmf.plot()
        #plt.plot(kmf.survival_function_.index.values, kmf.survival_function_.KM_estimate,ls='solid',c='C'+str(i))

N1 = ' N= ' + str(len(T_train[one_year_survival_prob_train >= one_yr_median_train ]))
N2 = ' N= ' + str(len(T_train[one_year_survival_prob_train < one_yr_median_train ]))

plt.xticks(np.arange(0, days_plot, 365))
plt.yticks(np.arange(0, 1.125, 0.125))
plt.xlim([0,days_plot])
plt.ylim([0,1])
plt.xlabel('Follow-up time (days)')
plt.ylabel('Probability of survival')
plt.legend(['Low Risk Individuals' + N1 ,'High Risk Individuals' + N2 ])
#	'1: Actual','1: Predicted')
# 	'2: Actual','2: Predicted',
# 	'3: Actual','3: Predicted',
# 	'4: Actual','4: Predicted'])
plt.title('Train set Kaplan-Meier Curves', fontweight = 'bold')

In [ ]:
one_year_survival_prob_all = np.cumprod(y_pred_overall[:,0:np.nonzero(breaks>365)[0][0]], axis=1)[:,-1]
one_yr_median_all = np.median(one_year_survival_prob_all)

In [ ]:
t[one_year_survival_prob_train >= one_yr_median_all]

In [ ]:
plt.figure(figsize=(8,6))
#plt.subplot(1, 2, 1)
days_plot = 365*9
for i in range(2):
    if i==0:
        kmf.fit(t[one_year_survival_prob_all >= one_yr_median_all ], event_observed=f[one_year_survival_prob_all >= one_yr_median_all])
        kmf.plot()
        #plt.plot(kmf.survival_function_.index.values, kmf.survival_function_.KM_estimate,ls='solid',c='C'+str(i))
	#pred_surv=np.mean(np.cumprod(y_pred_valid[F_test==i,:], axis=1),axis=0)
	#plt.plot(breaks,np.concatenate(([1],pred_surv)),ls='--',c='C'+str(i))
    else:
        kmf.fit(t[one_year_survival_prob_all < one_yr_median_all ], event_observed=f[one_year_survival_prob_all < one_yr_median_all])
        kmf.plot()
        #plt.plot(kmf.survival_function_.index.values, kmf.survival_function_.KM_estimate,ls='solid',c='C'+str(i))

N1 = ' N= ' + str(len(T_train[one_year_survival_prob_train >= one_yr_median_train ]))
N2 = ' N= ' + str(len(T_train[one_year_survival_prob_train < one_yr_median_train ]))

plt.xticks(np.arange(0, days_plot, 365))
plt.yticks(np.arange(0, 1.125, 0.125))
plt.xlim([0,days_plot])
plt.ylim([0,1])
plt.xlabel('Follow-up time (days)')
plt.ylabel('Probability of survival')
plt.legend(['Low Risk Individuals' + N1 ,'High Risk Individuals' + N2 ])
#	'1: Actual','1: Predicted')
# 	'2: Actual','2: Predicted',
# 	'3: Actual','3: Predicted',
# 	'4: Actual','4: Predicted'])
plt.title('Train set Kaplan-Meier Curves', fontweight = 'bold')

In [ ]:

plt.savefig('Kaplan-meier.png', dpi = 1200, format = 'png')

In [ ]:
five_year_survival_prob_train = np.cumprod(y_pred[:,0:np.nonzero(breaks>1875)[0][0]], axis=1)[:,-1]
five_year_survival_prob_valid = np.cumprod(y_pred_valid[:,0:np.nonzero(breaks>1875)[0][0]], axis=1)[:,-1]

In [ ]:
five_yr_median_train = np.median(five_year_survival_prob_train)
five_yr_median_valid = np.median(five_year_survival_prob_valid)

In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(1, 2, 1)
days_plot = 365*9
for i in range(2):
    if i==0:
        kmf.fit(T_train[five_year_survival_prob_train>=five_yr_median_train ],event_observed=F_train[five_year_survival_prob_train >=five_yr_median_train])
        #kmf.plot()
        plt.plot(kmf.survival_function_.index.values, kmf.survival_function_.KM_estimate,ls='solid',c='C'+str(i))
	#pred_surv=np.mean(np.cumprod(y_pred_valid[F_test==i,:], axis=1),axis=0)
	#plt.plot(breaks,np.concatenate(([1],pred_surv)),ls='--',c='C'+str(i))
    else:
        kmf.fit(T_train[five_year_survival_prob_train <five_yr_median_train ],event_observed=F_train[five_year_survival_prob_train<five_yr_median_train])
        #kmf.plot()
        plt.plot(kmf.survival_function_.index.values, kmf.survival_function_.KM_estimate,ls='solid',c='C'+str(i))

plt.xticks(np.arange(0, days_plot, 365))
plt.yticks(np.arange(0, 1.001, 0.125))
plt.xlim([0,days_plot])
plt.ylim([0,1])
plt.xlabel('Follow-up time (days)')
plt.ylabel('Probability of survival')
plt.legend(['Low Risk Individuals','High Risk Individuals'])
#	'1: Actual','1: Predicted')
# 	'2: Actual','2: Predicted',
# 	'3: Actual','3: Predicted',
# 	'4: Actual','4: Predicted'])
plt.title('Train set Kaplan-Meier Curves')
#plt.show()
plt.subplot(1, 2, 2)
days_plot = 365*9
for i in range(2):
    if i==0:
        kmf.fit(T_test[five_year_survival_prob_valid>=five_yr_median_valid ],event_observed=F_test[five_year_survival_prob_valid >=five_yr_median_valid])
        #kmf.plot()
        plt.plot(kmf.survival_function_.index.values, kmf.survival_function_.KM_estimate,ls='solid',c='C'+str(i))
	#pred_surv=np.mean(np.cumprod(y_pred_valid[F_test==i,:], axis=1),axis=0)
	#plt.plot(breaks,np.concatenate(([1],pred_surv)),ls='--',c='C'+str(i))
    else:
        kmf.fit(T_test[five_year_survival_prob_valid <five_yr_median_valid ],event_observed=F_test[five_year_survival_prob_valid<five_yr_median_valid])
        #kmf.plot()
        plt.plot(kmf.survival_function_.index.values, kmf.survival_function_.KM_estimate,ls='solid',c='C'+str(i))

plt.xticks(np.arange(0, days_plot, 365))
plt.yticks(np.arange(0, 1.0001, 0.125))
plt.xlim([0,days_plot])
plt.ylim([0,1])
plt.xlabel('Follow-up time (days)')
plt.ylabel('Probability of survival')
plt.legend(['Low Risk Individuals','High Risk Individuals'])
#	'1: Actual','1: Predicted')
# 	'2: Actual','2: Predicted',
# 	'3: Actual','3: Predicted',
# 	'4: Actual','4: Predicted'])
plt.title('Train set Kaplan-Meier Curves')
plt.show()

In [ ]:
#print(kmf.survival_function_)

In [ ]:
#print(kmf._median)

In [ ]:
# kmf.fit(t[g=='female'], event_observed=f[g=='female'])
# kmf.plot()

In [ ]:
# for i in ['male','female']:
# 	kmf.fit(t[g==i], event_observed=f[g==i])
#     #kmf.plot()
# 	plt.plot(kmf.survival_function_.index.values, kmf.survival_function_.KM_estimate,ls='-')
# 	#pred_surv=np.mean(np.cumprod(y_pred_valid[F_test==i,:], axis=1),axis=0)
# 	#predicted.append(plt.plot(breaks,np.concatenate(([1],pred_surv)),ls='--',c='C'+str(i)))

In [ ]:
actual.append(plt.plot(kmf.survival_function_.index.values, kmf.survival_function_.KM_estimate,ls='solid',c='C'+str(1)))

In [ ]:
pred_surv=np.mean(np.cumprod(y_pred[F_train==0,:], axis=1),axis=0)
pred_surv

In [ ]:
np.concatenate(([1],pred_surv))

In [ ]:
#predicted.append(plt.plot(breaks,np.concatenate(([1],pred_surv)),ls='-',c='C'+str(1)))

In [ ]:
days_plot = 365*10
for i in range(2):
	#kmf.fit(T_train[F_train==i], event_observed=F_train[F_train==i])
	#actual.append(plt.plot(kmf.survival_function_.index.values, kmf.survival_function_.KM_estimate,ls='--',c='C'+str(i)))
	pred_surv=np.mean(np.cumprod(y_pred_valid[F_test==i,:], axis=1),axis=0)
	plt.plot(breaks,np.concatenate(([1],pred_surv)),ls='--',c='C'+str(i))
plt.xticks(np.arange(0, days_plot, 365))
plt.yticks(np.arange(0, 1.0001, 0.125))
plt.xlim([0,days_plot])
plt.ylim([0,1])
plt.xlabel('Follow-up time (days)')
plt.ylabel('Probability of survival')
# plt.legend(['0: Actual','0: Predicted',
# 	'1: Actual','1: Predicted',
# 	'2: Actual','2: Predicted',
# 	'3: Actual','3: Predicted',
# 	'4: Actual','4: Predicted'])
plt.title('Test set calibration')
plt.show()


In [ ]:
temp1 = np.mean(np.cumprod(y_pred_valid[F_test==0,:], axis=1),axis=0)
temp1

In [ ]:
temp1concat = np.concatenate(([1],temp1))
temp1concat

In [ ]:
temp2 = np.mean(np.cumprod(y_pred_valid[F_test==1,:], axis=1),axis=0)
temp2

In [ ]:
temp2concat = np.concatenate(([1],temp2))
temp2concat

In [ ]:
len(temp2concat)

In [ ]:
# #model----------------------------------------------------------------------------------------------- toy model
# model = Sequential()
# model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
# #model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# #model.add(Dropout(0.25))
# model.add(Flatten())
# model.add(Dense(4, activation='relu'))
# prop_hazards=1
# if prop_hazards:
# 	model.add(Dense(1, use_bias=0, kernel_initializer='zeros'))
# 	model.add(nnet_survival.PropHazards(n_intervals))
# else:
# 	model.add(Dense(n_intervals, kernel_initializer='zeros', bias_initializer='zeros'))
# 	model.add(Activation('sigmoid'))

In [ ]:
#model.summary()

In [ ]:
model.compile(loss=nnet_survival.surv_likelihood(n_intervals), optimizer=optimizers.RMSprop(), metrics = ['accuracy'])

In [ ]:
#history=model.fit_generator(training_set, y_train, epochs=100)

In [ ]:
#np.where(event==1)

In [ ]:
# N_train = len(survival_times)
# R_matrix_train = np.zeros([N_train, N_train], dtype=int)

In [ ]:
# for i in range(N_train):
#         for j in range(N_train):
#             #print (survival_times[j],survival_times[i])
#             R_matrix_train[i,j] = survival_times[j] >= survival_times[i]

In [ ]:
# R_matrix_train

In [ ]:
# train_R = np.asarray(R_matrix_train,dtype=np.float32)
# train_ystatus = np.asarray(event,dtype=np.float32)

In [ ]:
# train_ystatus[0]

In [ ]:
# import sys
# sys.getsizeof(R_matrix_train)

In [ ]:
theta = lbl_pred.reshape(-1)
exp_theta = torch.exp(theta)

In [ ]:
# theta = lbl_pred.reshape(-1)
# exp_theta = torch.exp(theta)

In [ ]:
# n_intervals = 0

In [ ]:
input_shape = (122, 122, 1)

In [ ]:
# #model
# model = Sequential()
# model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
# #model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# #model.add(Dropout(0.25))
# model.add(Flatten())
# model.add(Dense(4, activation='relu'))
# prop_hazards=1
# if prop_hazards:
# 	model.add(Dense(1, use_bias=0, kernel_initializer='zeros'))
# 	model.add(nnet_survival.PropHazards(n_intervals))
# else:
# 	model.add(Dense(n_intervals, kernel_initializer='zeros', bias_initializer='zeros'))
# 	model.add(Activation('sigmoid'))

In [ ]:
#Trying to convert PropHazards model of CoxNN and SALMON converted to Keras; outputs prognostics index-------------- yet to succed
# model = Sequential()
# model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
# model.add(Flatten())
# model.add(Dense(4, activation='relu'))
# #model.add(Dense(1, use_bias=0, kernel_initializer='zeros'))
# #model.add(nnet_survival.PropHazards(1))


In [ ]:
model.summary()

In [ ]:
input_cox = model.layers[-1].output

In [ ]:
model.layers[-1].output_shape[1]

In [ ]:
model_cox = model.add(CoxRegression(input_cox, n_in=model.layers[-1].output_shape[1]))

In [ ]:
model.summary()

In [ ]:
model.layers[-1].output

In [ ]:
last_layer_output_dim

In [ ]:
last_layer_out,last_layer_output_dim = model.layers[-1].output,model.layers[-1].output_dim

In [ ]:
last_layer_out

In [ ]:
#type(train_ystatus[0])

In [ ]:
theta = K.reshape(last_layer_out, K.shape(last_layer_out))
theta2 = K.reshape(last_layer_out, K.shape(last_layer_output_dim))

In [ ]:
theta

In [ ]:
theta2

In [ ]:
# class CoxRegression(object):
#     def __init__(self, input, n_in):
#         self.W = theano.shared(value=numpy.zeros((n_in,1),dtype=theano.config.floatX), name='W_cox',borrow=True)
#         # b_values = numpy.zeros((1,), dtype=theano.config.floatX)
#         # self.b = theano.shared(value=b_values, name='b_cox', borrow=True) #intercept term is unnecessary
        
#         self.input = input[0] if len(input) == 1 else T.concatenate(input, axis=1)
#         #self.input = input

#         self.theta = T.dot(self.input, self.W) # + self.b
#         self.theta = T.reshape(self.theta, newshape=[T.shape(self.theta)[0]]) #recast theta as vector
#         self.exp_theta = T.exp(self.theta)
#         self.params = [self.W] #, self.b]

#     def negative_log_likelihood(self, R_batch, ystatus_batch):
#         return(-T.mean((self.theta - T.log(T.sum(self.exp_theta * R_batch,axis=1))) * ystatus_batch)) #exp_theta * R_batch ~ sum the exp_thetas of the patients with greater time e.g., R(t)
#         #e.g., all columns of product will have same value or zero, then do a rowSum
    
#     def evalNewData(self, test_data):
#         return(T.dot(T.concatenate(test_data, axis=1), self.W)) # + self.b)

In [ ]:
class CoxRegression(Layer):
    def __init__(self, input, n_in):
        #self.W = tf.Variable(value=numpy.zeros((n_in,1),dtype=K.float), name='W_cox',borrow=True)
        self.W = K.variable(value=np.zeros((n_in,1)), dtype='float32', name='W_cox', constraint=None)
        # b_values = numpy.zeros((1,), dtype=theano.config.floatX)
        # self.b = theano.shared(value=b_values, name='b_cox', borrow=True) #intercept term is unnecessary
        
        #self.input = input[0] if input.shape[1] == 1 else K.concatenate(input, axis=1)
        #self.input =  K.concatenate(input, axis=1)
        self.input = input

        self.theta = K.dot(self.input, self.W) # + self.b
        self.theta = K.reshape(self.theta, K.shape(self.theta)) #recast theta as vector
        self.exp_theta = K.exp(self.theta)
        self.params = [self.W] #, self.b]

    def negative_log_likelihood(self, R_batch, ystatus_batch):
        return(-K.mean((self.theta - K.log(K.sum(self.exp_theta * R_batch,axis=1))) * ystatus_batch)) #exp_theta * R_batch ~ sum the exp_thetas of the patients with greater time e.g., R(t)
        #e.g., all columns of product will have same value or zero, then do a rowSum
    
    def evalNewData(self, test_data):
        return(K.dot(K.concatenate(test_data, axis=1), self.W)) # + self.b)

In [ ]:
#K.is_keras_tensor(last_layer_out)

In [ ]:
#import tensorflow as tf
#@tf.function
def negative_log_likelihood(self,last_layer_out, R_batch, ystatus_batch):
    self.theta = K.reshape(last_layer_out, K.shape(last_layer_out))
    self.exp_theta = K.exp(self.theta)
    return(-K.mean((theta - K.log(K.sum(exp_theta * R_batch,axis=1))) * ystatus_batch))

In [ ]:
#tf.config.experimental_run_functions_eagerly(True)

In [ ]:
#model.compile(loss=negative_log_likelihood(last_layer_out, train_R, train_ystatus), optimizer=optimizers.Adam(), metrics = ['accuracy'])
#model.compile(loss=nnet_survival.surv_likelihood_edit(1,train_R, train_ystatus), optimizer=optimizers.Adam(), metrics = ['accuracy'])

In [ ]:
model.compile(loss=nnet_survival.surv_likelihood(n_intervals), optimizer=optimizers.Adam())

In [ ]:
#model.compile(loss=nnet_survival.surv_likelihood(n_intervals), optimizer=optimizers.Adam(), metrics = ['accuracy'])
#early_stopping = EarlyStopping(monitor='loss', patience=50)
#history=model.fit(x_train, y_train_array, batch_size=16, epochs=10, verbose=1, callbacks=[early_stopping], validation_split=0.33)

In [ ]:
history=model.fit(x_train, y_train_array, batch_size=1, epochs=10, verbose=1,validation_split=0.33)

In [ ]:
#Training set results
y_pred=model.predict_proba(x_train,verbose=0)


In [ ]:
y_pred.shape

In [ ]:
y_pred[:]

In [ ]:
 _ = np.nonzero(breaks>2)[0][0]
 _

In [ ]:
n_intervals

In [ ]:
#discrimination (C-index)
#oneyr_surv=np.cumprod(y_pred[:,0:np.nonzero(breaks>365)[0][0]], axis=1)[:,-1]
oneyr_surv=np.cumprod(y_pred[:,0:np.nonzero(breaks>2)[0][0]], axis=1)[:,-1]
print(concordance_index(time,oneyr_surv,event))

In [ ]:
#calibration plot
days_plot = 365*5
plt.figure(figsize=(5,5))
#plt.subplot(1, 2, 1)
kmf = KaplanMeierFitter()
matplotlib.style.use('default')
actual = []
predicted = []
for i in range(num_classes):
	kmf.fit(time[y_train==i], event_observed=event[y_train==i])
	actual.append(plt.plot(kmf.survival_function_.index.values, kmf.survival_function_.KM_estimate,ls='--',c='C'+str(i)))
	pred_surv=np.mean(np.cumprod(y_pred[y_train==i,:], axis=1),axis=0)
	predicted.append(plt.plot(breaks,np.concatenate(([1],pred_surv)),ls='-',c='C'+str(i)))
	#print(i, kmf.median_)

plt.xticks(np.arange(0, days_plot+0.0001, 200))
plt.yticks(np.arange(0, 1.0001, 0.125))
plt.xlim([0,days_plot])
plt.ylim([0,1])
plt.xlabel('Follow-up time (days)')
#plt.ylabel('Proportion surviving')
plt.ylabel('Probability of survival')
plt.legend(['0: Actual','0: Predicted',
	'1: Actual','1: Predicted',
	'2: Actual','2: Predicted',
	'3: Actual','3: Predicted',
	'4: Actual','4: Predicted'])
plt.title('Training set calibration')

In [ ]:
#Test set results
y_pred=model.predict_proba(x_test,verbose=0)

In [ ]:
#discrimination (C-index)
oneyr_surv=np.cumprod(y_pred[:,0:np.nonzero(breaks>365)[0][0]], axis=1)[:,-1]
print(concordance_index(timeTest,oneyr_surv,eventTest))

In [ ]:

#discrimination of perfect model that uses actual digit as survival time predictor
print(concordance_index(timeTest,-y_test.astype('float'),eventTest))

In [ ]:
#calibration
plt.figure(figsize=(10,5))
#plt.subplot(1, 2, 2)
kmf = KaplanMeierFitter()
matplotlib.style.use('default')
actual = []
predicted = []
for i in range(num_classes):
	kmf.fit(timeTest[y_test==i], event_observed=eventTest[y_test==i])
	actual.append(plt.plot(kmf.survival_function_.index.values, kmf.survival_function_.KM_estimate,ls='--',c='C'+str(i)))
	pred_surv=np.mean(np.cumprod(y_pred[y_test==i,:], axis=1),axis=0)
	predicted.append(plt.plot(breaks,np.concatenate(([1],pred_surv)),ls='-',c='C'+str(i)))

plt.xticks(np.arange(0, days_plot+0.0001, 200))
plt.yticks(np.arange(0, 1.0001, 0.125))
plt.xlim([0,days_plot])
plt.ylim([0,1])
plt.xlabel('Follow-up time (days)')
plt.ylabel('Probability of survival')
plt.legend(['0: Actual','0: Predicted',
	'1: Actual','1: Predicted',
	'2: Actual','2: Predicted',
	'3: Actual','3: Predicted',
	'4: Actual','4: Predicted'])
plt.title('Test set calibration')
plt.show()

In [ ]:
halflife=365.*0.7
breaks=-np.log(1-np.arange(0.0,0.96,0.1))*halflife/np.log(2) 




In [ ]:
n_intervals=len(breaks)-1
timegap = breaks[1:] - breaks[:-1]

In [ ]:
#y_train_array=nnet_survival.make_surv_array(time,event,breaks)

In [ ]:
# fixed_interval_width = 0
# if fixed_interval_width:
# 	breaks=np.arange(0.,365.*5,365./8)
# 	n_intervals=len(breaks)-1
# 	timegap = breaks[1:] - breaks[:-1]
# else:
# 	halflife=365.*5
# 	breaks=-np.log(1-np.arange(0.0,0.96,0.05))*halflife/np.log(2) 
# 	n_intervals=len(breaks)-1
# 	timegap = breaks[1:] - breaks[:-1]

In [ ]:
# import keras
# from keras.datasets import mnist

# num_classes = 2
# img_rows, img_cols = 28, 28

# (x_train, y_train), (x_test, y_test) = mnist.load_data()
# if K.image_data_format() == 'channels_first':
# 	x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
# 	x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
# 	input_shape = (1, img_rows, img_cols)
# else:
# 	x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
# 	x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
# 	input_shape = (img_rows, img_cols, 1)

# x_train = x_train.astype('float32')
# x_test = x_test.astype('float32')
# x_train /= 255
# x_test /= 255

# #Only keep numbers 0-4
# #Only keep numbers 0-1
# x_train = x_train[y_train<2, :, :, :]
# x_test = x_test[y_test<2, :, :, :]
# y_train = y_train[y_train<2]
# y_test = y_test[y_test<2]

# #Create simulated survival data, with higher numbers having shorter average survival
# sampleSizeTrain = x_train.shape[0]
# sampleSizeTest = x_test.shape[0]
# np.random.seed(0)
# beta = 0.9
# lambdaT = 365./np.log(2)
# lambdaC = 2*365./np.log(2)
# trueTime = np.random.exponential(scale = lambdaT * np.exp(-(beta*y_train)),size=sampleSizeTrain)
# #median surv is: 365.*np.exp(-beta*np.array([0,1,2,3,4]))
# #[365., 148.39792581,  60.3340942 ,  24.53001215, 9.97315869]
# censoringTime = np.random.exponential(scale = lambdaC, size=sampleSizeTrain)
# time = np.minimum(trueTime, censoringTime)
# event = (time == trueTime)*1.


# #for test data-----------------------------------------------------------------------------------------
# # trueTimeTest = np.random.exponential(scale = lambdaT * np.exp(-(beta*y_test)),size=sampleSizeTest)
# # censoringTimeTest = np.random.exponential(scale = lambdaC, size=sampleSizeTest)
# # timeTest = np.minimum(trueTimeTest, censoringTimeTest)
# # eventTest = (timeTest == trueTimeTest)*1.

# #Convert event data to array format

# #halflife=365.*0.7
# #breaks=-np.log(1-np.arange(0.0,0.96,0.1))*halflife/np.log(2) 
# #breaks=np.concatenate((np.arange(0,200,10),np.arange(200,1001,25)))

# #n_intervals=len(breaks)-1
# #timegap = breaks[1:] - breaks[:-1]
# #y_train_array=nnet_survival.make_surv_array(time,event,breaks)

# #Train model
# from numpy.random import seed
# seed(1)
# # from tensorflow import set_random_seed
# # set_random_seed(1)
# import tensorflow
# tensorflow.random.set_seed(1)

In [ ]:
#tf.config.experimental_run_functions_eagerly(True)

In [ ]:
#from keras.preprocessing.image import ImageDataGenerator
#training_list.sort()
#len(training_list)

In [ ]:
#train_datagen = ImageDataGenerator(rescale = 1./255)
#test_datagen = ImageDataGenerator(rescale = 1./255)
#train_datagen.fit(X_train)
#test_datagen.fit(X_test)